In [2]:
import torch

In [71]:
import torch.nn as nn

In [136]:
from torch.utils.data import Dataset,DataLoader,random_split

In [142]:
GPU = "cuda" if torch.cuda.is_available() else "cpu"

In [54]:
import random
import os
from tqdm import tqdm
from typing import *
from collections import defaultdict
import math
import string
from nltk.stem import PorterStemmer

import numpy as np

from predeal_dataset import *

In [204]:
EMB_DIM = 50

EPOCH = 100
BATCH_SIZE = 10 
LR = 0.002
L2 = 0.0001


In [145]:
def log_text_to_file(text:str)->None:
    with open("./output/log.txt","a") as fout:
        fout.write(text+"\n")

In [3]:
def load_GLoVe_embedding(dim:int=50)->Dict[str,List[float]]:
    res = dict()
    error_cnt = 0
    with open(f"./glove.6B/glove.6B.{dim}d.txt",encoding="utf-8") as fin:
        lines = fin.readlines()
        for line in tqdm(lines):
            elements = line.split()
            if len(elements)!=dim+1:
                error_cnt+=1
                continue
            word = elements[0]
            vector = list(map(float,elements[1:]))
            res[word]=vector
        print(f"{error_cnt} lines are discarded")
    return res

glove_emb_50d = load_GLoVe_embedding(EMB_DIM)

100%|██████████| 400000/400000 [00:10<00:00, 38963.76it/s]


0 lines are discarded


In [40]:
puncts = string.punctuation
stemmer = PorterStemmer()
def tokenize(sentence:str,use_stemmer:bool=True)->List[str]:
    # sentence = sentence.replace("(","( ").replace("[","[ ").replace("{","{ ")
    for p in puncts:
        sentence = sentence.replace(p,p+" ")
    res = list()
    tmp = sentence.split()
    for word in tmp:
        if len(word)==0:
            continue
        if word=="...":
            res.append(word)
            continue        
        if word[-1] in puncts:
            p = word[-1]
            word = word[:-1]
            if len(word)>0:
                word = word.lower()
                if use_stemmer:
                    word = stemmer.stem(word)
                res.append(word)
            res.append(p)
        else:
            word = word.lower()
            if use_stemmer:
                word = stemmer.stem(word)
            res.append(word)
    return res

In [36]:
spoiler_dataset = sample_sub_spoiler_set(SUBSET_SENTENCE_CNT)

In [41]:
word_cnt = defaultdict(int)

for datum in tqdm(spoiler_dataset):
    sentence = datum['review_sentence']
    for word in tokenize(sentence,False):
        word_cnt[word]+=1

100%|██████████| 50000/50000 [00:05<00:00, 8509.67it/s]


In [50]:
DICTIONARY_SIZE = 5000
SEPERATOR_SIGN = "seperatorsign"
UNKNOWN_WORD = "unknownword"

word_with_freq = list(word_cnt.items())
word_with_freq.sort(key=lambda tup:tup[1],reverse=True)
dictionary = word_with_freq[:]
dictionary = list(map(lambda tup:tup[0],dictionary))
dictionary = list(filter(lambda word:word in glove_emb_50d,dictionary))
dictionary = dictionary[:DICTIONARY_SIZE-2]
dictionary.append(SEPERATOR_SIGN)
dictionary.append(UNKNOWN_WORD)

word2id = {word:i for i,word in enumerate(dictionary)}

In [51]:
embedding = list(glove_emb_50d[word] for word in dictionary[:-2])

In [57]:
def norm(vector:List[float])->float:
    sq_sum = 0.0
    for x in vector:
        sq_sum+=x**2
    return sq_sum**0.5

emb_avg_norm = np.average([norm(v) for v in embedding])

In [63]:
def gen_random_emb():    
    res = np.random.randn(EMB_DIM)
    return (emb_avg_norm/norm(res))*res

In [67]:
embedding.append(gen_random_emb())
embedding.append(gen_random_emb())

In [69]:
embedding_tensor = torch.FloatTensor(embedding)

C:\Users\JipingZhang\AppData\Local\Temp\ipykernel_18232\3915031918.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  embedding_tensor = torch.FloatTensor(embedding)


In [74]:
unk_id = word2id[UNKNOWN_WORD]

def parse_id_sequence(sentence):
    res = list()
    words = tokenize(sentence,False)
    for word in words:
        if word not in word2id:
            res.append(unk_id)
        else:
            res.append(word2id[word])
    return res

In [206]:
class BiLSTMClassifierWithPretrainedEmbedding(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim:int=2*EMB_DIM, output_dim:int=2, n_layers:int=2):
        super().__init__()
        embedding_dim = embedding_matrix.size(1)
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, 
                            bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2+1, output_dim)
        

    def forward(self, text):
        embedded = self.embedding(text)
        l = len(embedded)
        l_tensor = torch.FloatTensor([l]).to(GPU)
        outputs, (hidden, cell) = self.lstm(embedded)
        outputs = outputs[-1,:]
        outputs_with_l = torch.concat((outputs,l_tensor))
        dense_outputs = self.fc(outputs_with_l)
        return dense_outputs

In [144]:
model = BiLSTMClassifierWithPretrainedEmbedding(embedding_tensor)

In [154]:
model.to("cpu")
input_ids = parse_id_sequence("can can need new new")
input_tensor = torch.LongTensor(input_ids)
input_tensor = input_tensor.reshape((-1,1))
loss_func = nn.CrossEntropyLoss()
output = model(input_tensor)
print(output)
loss_func(output,torch.LongTensor([1])[0])

tensor([ 0.0126, -0.0409], grad_fn=<AddBackward0>)


tensor(0.7203, grad_fn=<NllLossBackward0>)

In [138]:
class ListDataset(Dataset):
    def __init__(self,*lists) -> None:
        super().__init__()
        if len(lists)==0:
            raise ValueError("Expecting at least one list")
        l = len(lists[0])
        for i,li in enumerate(lists):
            if not isinstance(li,(list,tuple,np.ndarray,torch.Tensor)):
                raise ValueError(f"expecting input to be list,tuple,numpy-array or torch's tensor, actually get {type(li)} at {i}-th argument")
            if len(li)!=l:
                raise ValueError(f"length of {i}-th argument is {len(li)}, length of 0-th argument is {l}, they don't match")
        self.lists = lists
        self.l = l
    
    def __len__(self):
        return self.l
    
    def __getitem__(self, index) -> Any:
        return tuple(map(lambda l:l[index],self.lists))

def get_spoiler_dataset(spoiler_dataset_raw:List[dict])->Dataset:
    xs = list()
    ys = list()
    for datum in spoiler_dataset_raw:
        xs.append(datum['review_sentence'])
        ys.append(datum['label'])
    return ListDataset(xs,ys)

spoiler_dataset_processed = get_spoiler_dataset(spoiler_dataset)

In [140]:
train_dataset, valid_dataset, test_dataset = random_split(spoiler_dataset_processed, [TRAIN_SET_CNT, VALID_SET_CNT, TEST_SET_CNT])

In [198]:
def get_performance_info(y_actual,y_predict):
    y_actual = np.array(y_actual)
    y_predict = np.array(y_predict)
    y_actual = y_actual.reshape((-1,))
    y_predict = y_predict.reshape((-1,))
    TP = np.sum((y_actual == 1) & (y_predict == 1))
    FP = np.sum((y_actual == 0) & (y_predict == 1))
    TN = np.sum((y_actual == 0) & (y_predict == 0))
    FN = np.sum((y_actual == 1) & (y_predict == 0))
    TPR = TP / (TP + FN)
    FPR = FP / (FP + TN)
    TNR = TN / (TN + FP)
    FNR = FN / (TP + FN)
    BER = 1 - (0.5 * (TPR + TNR))
    accu = np.sum(y_actual==y_predict)/len(y_actual)
    return accu,TP,FP,TN,FN,TPR, FPR, TNR, FNR, BER

e = 2.718281828

def get_best_ths_with_ber(pred_prop_with_label): 
    pred_prop_with_label.sort(reverse=True)
    valid_set_pos_cnt = sum(tup[1] for tup in pred_prop_with_label)
    valid_set_neg_cnt = len(pred_prop_with_label)-valid_set_pos_cnt
    best_ths = 1.0
    best_ber = 0.5
    curr_false_positive = 0
    curr_false_negative = valid_set_pos_cnt
    for (prob,label) in pred_prop_with_label:
        ths = prob-0.00001
        if label==1:
            curr_false_negative-=1
        else:
            curr_false_positive+=1
        ber = 0.5*(curr_false_negative/valid_set_pos_cnt+curr_false_positive/valid_set_neg_cnt)
        if ber<best_ber:
            best_ber = ber
            best_ths = ths
    return best_ths,best_ber

def evaluate_dynamic_prob_ths(model,dataset):
    model.to(GPU)
    e = 2.718281828
    y_pred_logits = list()
    y_label = list()
    with torch.no_grad():
        for b_x,b_y in tqdm(dataset):
            input_ids = torch.LongTensor(parse_id_sequence(b_x)).to(GPU)
            output = model(input_ids).to("cpu").detach()
            y_label.append(b_y)
            y_pred_logits.append(output.numpy().tolist())
    y_pos_prob_pred = list(map(lambda logits:e**logits[1]/(e**logits[0]+e**logits[1]),y_pred_logits))
    pred_prob_with_label = list(zip(y_pos_prob_pred,y_label))
    judging_ths,ber = get_best_ths_with_ber(pred_prob_with_label)
    y_pred = list(int(p>judging_ths) for p in y_pos_prob_pred)
    return *get_performance_info(y_label,y_pred),judging_ths


In [172]:
model = BiLSTMClassifierWithPretrainedEmbedding(embedding_tensor)

# model.fc

evaluate_dynamic_prob_ths(model,valid_dataset)

  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:19<00:00, 261.96it/s]


(0.8102,
 51,
 696,
 4000,
 253,
 0.16776315789473684,
 0.14821124361158433,
 0.8517887563884157,
 0.8322368421052632,
 0.4902240428584237,
 0.49586474531550173)

In [181]:
class ShuffledDataset(Dataset):
    def __init__(self,dataset) -> None:
        super().__init__()
        self.dataset = dataset
        indices = list(range(len(dataset)))
        random.shuffle(indices)
        self.indices = indices

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index) -> Any:
        return self.dataset[self.indices[index]]

In [208]:
model = BiLSTMClassifierWithPretrainedEmbedding(embedding_tensor)
model.to(GPU)

pos_cnt = sum(y for (x,y) in train_dataset)
neg_cnt = len(train_dataset)-pos_cnt
weights = [pos_cnt/len(train_dataset),neg_cnt/len(train_dataset)]
weights = torch.tensor(weights, dtype=torch.float).to(GPU)
loss_func = torch.nn.CrossEntropyLoss(weights)
weight_params = [param for name, param in model.named_parameters() if 'weight' in name]
bias_params = [param for name, param in model.named_parameters() if 'bias' in name]
optimizer = torch.optim.Adam([
    {'params': weight_params, 'weight_decay': L2},
    {'params': bias_params, 'weight_decay': 0.0}  
], lr=LR/BATCH_SIZE) 
# optimizer = torch.optim.Adam(model.parameters(),LR/BATCH_SIZE,weight_decay=L2)

# data_loader = DataLoader(train_dataset,batch_size=1,shuffle=True)
log_text_to_file("start training bilstm model , dataset is review_text only")
for e in range(EPOCH):
    for step,(b_x,b_y) in enumerate(tqdm(ShuffledDataset(train_dataset))):
        input_ids = torch.LongTensor(parse_id_sequence(b_x)).to(GPU)
        output = model(input_ids)
        b_y = torch.LongTensor([b_y])[0].to(GPU)
        loss = loss_func(output,b_y)
        if (step+1)%BATCH_SIZE==0:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    accu,TP,FP,TN,FN,TPR, FPR, TNR, FNR, BER,ths = evaluate_dynamic_prob_ths(model,valid_dataset)
    print(f"ber after epoch {e+1}: {BER}")
    msg = "    %7s%7s%7s%7s%7s%7s%7s\n     %.4f %.4f %.4f %.4f %.4f %.4f %.4f"%("accu","ber","tpr","fpr","tnr","fnr","ths",accu,BER,TPR,FPR,TNR,FNR,ths)
    print(msg)
    save_path = f"./output/review_only_lstm_e{e}_b{BATCH_SIZE}_lr{LR}_l2{L2}_ber{BER:.4f}"
    torch.save(model.state_dict(),save_path)
    log_text_to_file(f"    ber after epoch {e+1}: {BER}, model params saved to {save_path}, other performance infos:{accu,TP,FP,TN,FN,TPR, FPR, TNR, FNR, BER}")
    log_text_to_file(msg)

100%|██████████| 5000/5000 [00:19<00:00, 257.01it/s]


ber after epoch 1: 0.36216152604680363
       accu    ber    tpr    fpr    tnr    fnr    ths
     0.6260 0.3622 0.6513 0.3756 0.6244 0.3487 0.0438


100%|██████████| 5000/5000 [00:19<00:00, 256.75it/s]


ber after epoch 2: 0.3543161032905945
       accu    ber    tpr    fpr    tnr    fnr    ths
     0.5136 0.3543 0.7961 0.5047 0.4953 0.2039 0.0510


100%|██████████| 5000/5000 [00:19<00:00, 254.91it/s]


...

ber after epoch 24: 0.3474457545055142
       accu    ber    tpr    fpr    tnr    fnr    ths
     0.5554 0.3474 0.7632 0.4580 0.5420 0.2368 0.0269


100%|██████████| 5000/5000 [00:19<00:00, 260.77it/s]


ber after epoch 25: 0.35121155742849464
       accu    ber    tpr    fpr    tnr    fnr    ths
     0.5830 0.3512 0.7237 0.4261 0.5739 0.2763 0.0472


  6%|▌         | 2223/40000 [00:10<03:03, 205.37it/s]


KeyboardInterrupt: 